In [1]:
import numpy as np
import skimage
import utils
import pathlib

In [2]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    threshold = 128

    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    L = 256 # Possible intensities
    MN = im.shape[0] * im.shape[1] # Total number of pixels in the image
    
    cum_sums = np.zeros(L) # Cummulative sums = sum(pi)
    cum_means = np.zeros(L) # Cummulative means = sum(i*pi), last value is the global mean
    cum_sums[0] =  np.count_nonzero(im == 0) / MN # The first cummulative sum is the value of the normalized histogram at 0
    for i in range(1, L):
        # Normalized histogram for value i, no need to save in array as it is only used here
        norm_hist_i = np.count_nonzero(im == i) / MN 
        cum_sums[i] = cum_sums[i-1] + norm_hist_i
        cum_means[i] = cum_means[i-1] + i*norm_hist_i

    max_between = 0 # Maximum between class variance, the Otsu optimal threeshold is the index for which we find this maximum
    for i in range(1, L):
        if(cum_sums[i] == 0): continue # No image values found yet, do not get division by 0
        # We find the between class variances for the index i and check if we have a new maximum, do not need to save in array
        # as it is only used here
        between_class_vars_i = (cum_means[L-1] * cum_sums[i] - cum_means[i])**2 / (cum_sums[i]*(1-cum_sums[i]))
        if(between_class_vars_i  > max_between):
            max_between =  between_class_vars_i 
            threshold = i
    
    ### END YOUR CODE HERE ###

    return threshold

In [3]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\polymercell.png


C:\Users\Pablo\AppData\Local\Temp\ipykernel_12108\1997791817.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(


Found optimal threshold: 181
Saving image to: image_processed\polymercell-segmented.png
